# TA Analysis

---
Goals:
1. Identify baseline vs model (baseline correct vs model false) instance
2. Check wether the problem in baseline still linger in model
3. Imply same rule for implicit aspect 


In [1]:
# === Importing Library ===

import pandas as pd
import pickle

In [2]:
# == Baseline reports ==
path = "./data/m3/test/triplets.csv"
triplets = pd.read_csv(path)
triplets = triplets.fillna("-")
triplets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2510 entries, 0 to 2509
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   idx                2510 non-null   int64 
 1   text               2510 non-null   object
 2   all_labels         2510 non-null   object
 3   all_preds          2510 non-null   object
 4   all_fixed_preds    2510 non-null   object
 5   matching_comments  2510 non-null   object
dtypes: int64(1), object(5)
memory usage: 117.8+ KB


In [3]:
# == Baseline reports ==
path = "./data/m3/test/result.csv"
results = pd.read_csv(path)
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   idx                1000 non-null   int64 
 1   text               1000 non-null   object
 2   all_labels         1000 non-null   object
 3   all_preds          1000 non-null   object
 4   all_fixed_preds    1000 non-null   object
 5   matching_comments  1000 non-null   object
 6   num_true           1000 non-null   int64 
 7   num_false          1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [4]:
triplets[triplets.idx==448]

,idx,text,all_labels,all_preds,all_fixed_preds,matching_comments
1145,448,"kamar luas dan rapi , tetapi sayang untuk tv k...","('kamar', 'luas', 'positif')","('kamar', 'luas', 'positif')","('kamar', 'luas', 'positif')",Found Exact
1146,448,"kamar luas dan rapi , tetapi sayang untuk tv k...","('kamar', 'rapi', 'positif')","('kamar', 'rapi', 'positif')","('kamar', 'rapi', 'positif')",Found Exact
1147,448,"kamar luas dan rapi , tetapi sayang untuk tv k...","('tv', 'jelek sinyalnya', 'negatif')","('tv', 'jelek sinyalnya', 'negatif')","('tv', 'jelek sinyalnya', 'negatif')",Found Exact
1148,448,"kamar luas dan rapi , tetapi sayang untuk tv k...","('wifi', 'tidak tersedia', 'negatif')","('wifi', 'tidak tersedia', 'negatif')","('wifi', 'tidak tersedia', 'negatif')",Found Exact


In [5]:
mask = results.num_false > 0
print(f"Total false sentence : {sum(mask)}")

Total false sentence : 309


In [6]:
false_idx = results[mask].idx

In [7]:
miss_predict = triplets.loc[(triplets.idx.isin(false_idx)) & (triplets.matching_comments != "Found Exact")]

print(f"Total false predicts triplets :{len(miss_predict)}")

Total false predicts triplets :527


In [8]:
miss_predict.head(10)

,idx,text,all_labels,all_preds,all_fixed_preds,matching_comments
0,0,"pelayanan ramah , kamar nyaman dan fasilitas l...","('airnya showernya', 'kurang panas', 'negatif')","('air showernya', 'kurang panas', 'negatif')","(', showernya', 'kurang panas', 'negatif')",Different Aspect Term
14,5,"room bersih tetapi sayang linen kotor , bekas ...","('handuk', 'putih kehitaman', 'negatif')","('handuk', 'kehitaman', 'negatif')","('handuk', 'kehitaman', 'negatif')",Different Sentiment Term
16,5,"room bersih tetapi sayang linen kotor , bekas ...","('linen', 'bekas ada spot', 'negatif')","('handuk', 'sudah waktunya di ganti', 'negatif')","('handuk', 'sudah waktunya di ganti', 'negatif')",Subset aspect/sentiment
19,5,"room bersih tetapi sayang linen kotor , bekas ...","('sinyal', 'ada', 'positif')",-,-,Not Generated
20,5,"room bersih tetapi sayang linen kotor , bekas ...","('sinyal', 'tidak mau terhubung', 'negatif')",-,-,Not Generated
42,12,acnya kurang dingin dan tv channel rata ngilang .,"('tv channel', 'ngilang', 'negatif')","('tv channel', 'rata ngilang', 'negatif')","('tv channel', 'rata ngilang', 'negatif')",Different Sentiment Term
43,13,"enak banget dapat sarapan , tetapi wifinya kur...","('sarapan', 'enak banget dapat', 'positif')","('sarapan', 'dapat', 'positif')","('sarapan', 'dapat', 'positif')",Different Sentiment Term
45,14,"pelayanan baik namun berkesan kotor , terutama...","('koridornya', 'kotor', 'negatif')","('pelayanan', 'berkesan kotor', 'negatif')","('pelayanan', 'berkesan kotor', 'negatif')",Subset aspect/sentiment
46,14,"pelayanan baik namun berkesan kotor , terutama...","('lobi', 'kotor', 'negatif')",-,-,Not Generated
52,16,kamar kurang terawat tetapi bagus lah pakai st...,"('stay transit', 'bagus', 'positif')","('kamar', 'bagus', 'positif')","('kamar', 'bagus', 'positif')",Different Aspect Term


In [9]:
miss_predict.tail(10)

,idx,text,all_labels,all_preds,all_fixed_preds,matching_comments
2440,968,"kamar bersih , pelayanan bagus . check in tida...",-,"('check in', 'tidak bisa', 'negatif')","('check in', 'tidak bisa', 'negatif')",No Label Matched
2447,970,"kamar kecil , tetapi nyaman , kamar mandi bers...",-,"('kamar mandi', 'nyaman', 'positif')","('kamar mandi', 'nyaman', 'positif')",No Label Matched
2457,973,"pelayanannya baik dan ramah , ada parkir kenda...",-,"('kamar', 'nyamuk', 'negatif')","('kamar', 'nyamuk', 'negatif')",No Label Matched
2458,974,lebih baik cari yang lain . lebih jam 11 tidak...,-,"('hotel', 'tidak bisa masuk', 'negatif')","('hotel', 'tidak bisa masuk', 'negatif')",No Label Matched
2465,977,pengalaman saya disini menyenangkan . tetapi d...,"('gagang pintu kamar mandi', 'tidak ada', 'neg...","(' gagang pintu kamar mandi', 'tidak ada', 'ne...","('. gagang pintu kamar mandi', 'tidak ada', 'n...",Different Aspect Term
2468,977,pengalaman saya disini menyenangkan . tetapi d...,-,"('pengalaman', 'menyenangkan', 'positif')","('pengalaman', 'menyenangkan', 'positif')",No Label Matched
2469,978,kamar airy diletakan dilantai 3 . tangga manua...,-,"('tangga', 'manual', 'negatif')","('tangga', 'manual', 'negatif')",No Label Matched
2470,979,ke lobi susah jika ada penitipan barang di baw...,-,"('penitipan barang', 'susah', 'negatif')","('penitipan barang', 'susah', 'negatif')",No Label Matched
2486,988,gambar kamarnya kurang sesuai dengan keadaan y...,"('kamarnya', 'kurang sesuai', 'negatif')","('gambar kamarnya', 'kurang sesuai', 'negatif')","('gambar kamarnya', 'kurang sesuai', 'negatif')",Different Aspect Term
2490,991,ranjang tidur yang sudah mulai rusak .,"('ranjang tidur', 'rusak', 'negatif')","('ranjang tidur', 'mulai rusak', 'negatif')","('ranjang tidur', 'mulai rusak', 'negatif')",Different Sentiment Term


In [10]:
miss_predict.to_csv("analysis.csv", index=False)

In [11]:
triplets.loc[triplets.idx==18]

,idx,text,all_labels,all_preds,all_fixed_preds,matching_comments
56,18,lokasi bagus tengah kota . kamar bersih hanya ...,"('akses internet', 'tidak ada', 'negatif')",-,-,Not Generated
57,18,lokasi bagus tengah kota . kamar bersih hanya ...,"('kamar', 'bersih', 'positif')","('kamar', 'bersih', 'positif')","('kamar', 'bersih', 'positif')",Found Exact
58,18,lokasi bagus tengah kota . kamar bersih hanya ...,"('kamar mandi', 'bau busuk', 'negatif')","('kamar mandi', 'bau busuk', 'negatif')","('kamar mandi', 'bau busuk', 'negatif')",Found Exact
59,18,lokasi bagus tengah kota . kamar bersih hanya ...,"('lift', 'tidak ada', 'negatif')","('lift', 'tidak ada', 'negatif')","('lift', 'tidak ada', 'negatif')",Found Exact
60,18,lokasi bagus tengah kota . kamar bersih hanya ...,"('lokasi', 'bagus', 'positif')","('lokasi', 'bagus', 'positif')","('lokasi', 'bagus', 'positif')",Found Exact
61,18,lokasi bagus tengah kota . kamar bersih hanya ...,"('makanan', 'biasa', 'negatif')","('makanan', 'biasa saja', 'negatif')","('makanan', 'biasa saja', 'negatif')",Different Sentiment Term
62,18,lokasi bagus tengah kota . kamar bersih hanya ...,"('staf recepsionis', 'tidak ramah', 'negatif')","('staf recepsionis', 'tidak ramah', 'negatif')","('staf recepsionis', 'tidak ramah', 'negatif')",Found Exact


Identify baseline vs model (baseline correct vs model false) instance

In [12]:
ta_gama = pd.read_csv("./data/ta-gama/test-report.csv")

In [13]:
indices = miss_predict.idx 
ta_gama_checking = ta_gama.loc[ta_gama.idx.isin(indices)].copy()

In [14]:
ta_gama_checking["model_preds"] = results.loc[results.idx.isin(indices)]["all_fixed_preds"]
ta_gama_checking["labels"] = results.loc[results.idx.isin(indices)]["all_labels"]

In [15]:
ta_gama_checking.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 304 entries, 0 to 991
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   idx          304 non-null    int64 
 1   sents        304 non-null    object
 2   triples      304 non-null    object
 3   model_preds  304 non-null    object
 4   labels       304 non-null    object
dtypes: int64(1), object(4)
memory usage: 14.2+ KB


In [16]:
polarity_annotation_map = {
    "NT": "netral",
    "NG": "negatif",
    "PO": "positif",
}
idx_sorting_fn = lambda tuplet: (tuplet[0], tuplet[1], tuplet[2])

def parse(instance):
    raw = eval(instance)
    triplets = []
    for triplet in raw:
        aspect = " ".join(triplet[0])
        sentiment = " ".join(triplet[1])
        polarity = polarity_annotation_map[triplet[2]]
        triplets.append((aspect, sentiment, polarity))
    triplets = sorted(triplets, key=idx_sorting_fn)
    return triplets

ta_gama_checking["triples"] = ta_gama_checking["triples"].apply(parse)

In [17]:
def count_correct(true, pred):
    true = set(eval(true))
    pred = set(pred)
    return len(true & pred) == len(pred) 

ta_gama_checking_mask = ta_gama_checking.apply(lambda row: count_correct(row["labels"], row["triples"]), axis = 1)

In [18]:
sum(ta_gama_checking_mask)

163

In [19]:
ta_gama_checking[ta_gama_checking_mask].to_csv("versus-old-exp.csv", index=False)

Type II Error

In [20]:
type2err = pd.read_csv("./data/ta-gama/type-2.csv")
type2err.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   idx       160 non-null    int64 
 1   sents     160 non-null    object
 2   triples   160 non-null    object
 3   filename  160 non-null    object
dtypes: int64(1), object(3)
memory usage: 5.1+ KB


In [21]:
type2err_mask = (ta_gama.idx.isin(type2err.idx)) & (ta_gama.idx.isin(results[results.num_false == 0].idx))
type2_ta_gama_checking = ta_gama[type2err_mask]
type2_ta_gama_checking["model_preds"] = results.loc[results.idx.isin(type2_ta_gama_checking.idx)]["all_fixed_preds"]
type2_ta_gama_checking["labels"] = results.loc[results.idx.isin(type2_ta_gama_checking.idx)]["all_labels"]
type2_ta_gama_checking["triples"] = type2_ta_gama_checking["triples"].apply(parse)
type2_ta_gama_checking.to_csv("type2-exp.csv", index=False)

C:\Users\William\AppData\Local\Temp\ipykernel_17820\2711524811.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  type2_ta_gama_checking["model_preds"] = results.loc[results.idx.isin(type2_ta_gama_checking.idx)]["all_fixed_preds"]
C:\Users\William\AppData\Local\Temp\ipykernel_17820\2711524811.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  type2_ta_gama_checking["labels"] = results.loc[results.idx.isin(type2_ta_gama_checking.idx)]["all_labels"]
C:\Users\William\AppData\Local\Temp\ipykernel_17820\271152

Implicit Aspect

In [22]:
# == Baseline reports ==
path = "./data/m3-implicit/test/triplets.csv"
triplets_implicit = pd.read_csv(path)
triplets_implicit = triplets_implicit.fillna("-")

path = "./data/m3-implicit/test/result.csv"
results_implicit = pd.read_csv(path)

In [23]:
sum(false_idx)

153770

In [24]:
false_idx = results_implicit.loc[results_implicit.num_false > 0].idx
miss_predict_implicit = triplets_implicit.loc[(triplets_implicit.idx.isin(false_idx)) & (triplets_implicit.matching_comments != "Found Exact")]

print(f"Total false predicts triplets :{len(miss_predict_implicit)}")

Total false predicts triplets :541


In [25]:
miss_predict_implicit

,idx,text,all_labels,all_preds,all_fixed_preds,matching_comments
0,0,"pelayanan ramah , kamar nyaman dan fasilitas l...","('airnya showernya', 'kurang panas', 'negatif')","('airnya', 'kurang panas', 'negatif')","('airnya', 'kurang panas', 'negatif')",Different Aspect Term
4,1,tidak terlalu jauh dari pusat kota .,"('dari pusat kota', 'tidak terlalu jauh', 'pos...","('hotel', 'tidak terlalu jauh', 'positif')","('hotel', 'tidak terlalu jauh', 'positif')",Different Aspect Term
14,5,"room bersih tetapi sayang linen kotor , bekas ...","('handuk', 'putih kehitaman', 'negatif')","('handuk', 'kehitaman', 'negatif')","('handuk', 'kehitaman', 'negatif')",Different Sentiment Term
16,5,"room bersih tetapi sayang linen kotor , bekas ...","('linen', 'bekas ada spot', 'negatif')","('handuk', 'sudah waktunya di ganti', 'negatif')","('handuk', 'sudah waktunya di ganti', 'negatif')",Subset aspect/sentiment
19,5,"room bersih tetapi sayang linen kotor , bekas ...","('sinyal', 'ada', 'positif')",-,-,Not Generated
...,...,...,...,...,...,...
2515,977,pengalaman saya disini menyenangkan . tetapi d...,-,"('pengalaman', 'menyenangkan', 'positif')","('pengalaman', 'menyenangkan', 'positif')",No Label Matched
2516,978,kamar airy diletakan dilantai 3 . tangga manua...,-,"('tangga', 'manual', 'negatif')","('tangga', 'manual', 'negatif')",No Label Matched
2517,979,ke lobi susah jika ada penitipan barang di baw...,-,"('lobi', 'susah', 'negatif')","('lobi', 'susah', 'negatif')",No Label Matched
2533,988,gambar kamarnya kurang sesuai dengan keadaan y...,"('kamarnya', 'kurang sesuai', 'negatif')","('gambar kamarnya', 'kurang sesuai', 'negatif')","('gambar kamarnya', 'kurang sesuai', 'negatif')",Different Aspect Term


In [26]:
with open("./data/annotation/test_skip_label.pkl", 'rb') as f:
    test_skip_label = pickle.load(f)

In [27]:
miss_predict_implicit.loc[miss_predict_implicit.idx.isin(test_skip_label)].to_csv("analysis-implicit.csv", index=False)

In [29]:
triplets_implicit.loc[triplets_implicit.idx==762]

,idx,text,all_labels,all_preds,all_fixed_preds,matching_comments
1963,762,"tetapi kurang bersih , masih ada sisa rambut d...","('hotel', 'kurang bersih', 'negatif')","('tempat tidur', 'kurang bersih', 'negatif')","('tempat tidur', 'kurang bersih', 'negatif')",Different Aspect Term
